In [1]:
! pip install pandas matplotlib python-docx gTTS yagmail PyMuPDF

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from docx import Document
from docx.shared import Inches
from gtts import gTTS
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import fitz
import os

In [7]:
data = {
    'nombre': ['Ana Gomez', 'Luis Perez', 'Sofia Ruiz', 'Carlos Soto'],
    'puesto': ['Gerente', 'Vendedor', 'Gerente', 'Vendedor'],
    'salario': [60000, 45000, 62000, 48000],
    'ventas': [120000, 150000, 135000, 165000],
    'kpi_comunicacion': [8, 7, 9, 8],
    'kpi_liderazgo': [9, 6, 8, 7],
    'kpi_adaptabilidad': [7, 8, 9, 7]
}
df = pd.DataFrame(data)
df.to_excel('empleados.xlsx', index=False)
print("Archivo 'empleados.xlsx' creado con exito.")

def requiere_revision(func):
    def wrapper(self, *args, **kwargs):
        print(f"ATENCION: El informe de {self.nombre} requiere una revision manual.")
        return func(self, *args, **kwargs)
    return wrapper

class Empleado:
    def __init__(self, nombre, puesto, salario, kpis):
        self.nombre = nombre
        self.puesto = puesto
        self.__salario = salario
        self.kpis = kpis

    def calcular_bono(self):
        return self.__salario * 0.10

class EmpleadoComercial(Empleado):
    def __init__(self, nombre, puesto, salario, kpis, ventas):
        super().__init__(nombre, puesto, salario, kpis)
        self.ventas = ventas

    @requiere_revision
    def calcular_bono(self):
        return (self.ventas * 0.05) if self.ventas > 150000 else 0

def leer_datos_empleados(archivo_excel):
    df = pd.read_excel(archivo_excel)
    empleados = []
    for _, row in df.iterrows():
        kpis = {
            'Comunicacion': row['kpi_comunicacion'],
            'Liderazgo': row['kpi_liderazgo'],
            'Adaptabilidad': row['kpi_adaptabilidad']
        }
        if row['puesto'] == 'Vendedor':
            empleados.append(EmpleadoComercial(row['nombre'], row['puesto'], row['salario'], kpis, row['ventas']))
        else:
            empleados.append(Empleado(row['nombre'], row['puesto'], row['salario'], kpis))
    return empleados

empleados = leer_datos_empleados('empleados.xlsx')
print("Datos de empleados leidos y objetos creados.")

Archivo 'empleados.xlsx' creado con exito.
Datos de empleados leidos y objetos creados.


In [8]:
def generar_grafico_radar(empleado, nombre_archivo):
    labels = list(empleado.kpis.keys())
    values = list(empleado.kpis.values())
    
    num_vars = len(labels)
    angles = [n / float(num_vars) * 2 * 3.14159 for n in range(num_vars)]
    angles += angles[:1]
    
    values += values[:1]

    plt.figure(figsize=(6, 6))
    ax = plt.subplot(111, polar=True)
    ax.set_theta_offset(3.14159 / 2)
    ax.set_theta_direction(-1)
    
    plt.xticks(angles[:-1], labels, color='grey', size=12)
    ax.set_rlabel_position(0)
    plt.yticks([2, 4, 6, 8, 10], ["2", "4", "6", "8", "10"], color="grey", size=10)
    plt.ylim(0, 10)
    
    ax.plot(angles, values, linewidth=2, linestyle='solid', label='KPIs de Desempeno')
    ax.fill(angles, values, 'b', alpha=0.1)
    
    plt.title(f'KPIs de {empleado.nombre}', size=15, color='blue', y=1.1)
    plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
    plt.savefig(nombre_archivo, dpi=100)
    plt.close()
    print(f"Grafico de radar para {empleado.nombre} creado.")

def crear_informe_word(empleado, nombre_archivo_word, nombre_archivo_png):
    doc = Document()
    doc.add_heading(f'Informe de Desempeno Anual - {empleado.nombre}', 0)
    doc.add_paragraph(f'Puesto: {empleado.puesto}')
    doc.add_paragraph(f'Salario Base: ${empleado._Empleado__salario}')
    
    doc.add_heading('Analisis de KPIs', level=1)
    doc.add_picture(nombre_archivo_png, width=Inches(5))
    
    resumen = (f'Hola {empleado.nombre}, tu desempeno ha sido sobresaliente. '
               f'Tu bono por rendimiento es de ${empleado.calcular_bono():,.2f}. '
               f'Tus metricas de KPIs son: {empleado.kpis}.')
    doc.add_paragraph(resumen)
    doc.save(nombre_archivo_word)
    print(f"Informe de Word para {empleado.nombre} creado.")

    tts = gTTS(text=resumen, lang='es')
    nombre_audio = nombre_archivo_word.replace('.docx', '.mp3')
    tts.save(nombre_audio)
    print(f"Resumen de audio para {empleado.nombre} creado.")
    return nombre_audio

In [10]:
def enviar_correo(destinatario, asunto, cuerpo, adjunto_word, adjunto_audio):
    remitente = 'darwinvillalobos0201@gmail.com'
    password = 'flka alne hpip pgwk'

    msg = MIMEMultipart()
    msg['From'] = remitente
    msg['To'] = destinatario
    msg['Subject'] = asunto
    
    msg.attach(MIMEText(cuerpo, 'plain'))

    with open(adjunto_word, 'rb') as f:
        part = MIMEBase('application', 'octet-stream')
        part.set_payload(f.read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', f'attachment; filename="{os.path.basename(adjunto_word)}"')
        msg.attach(part)
        
    with open(adjunto_audio, 'rb') as f:
        part = MIMEBase('audio', 'mp3')
        part.set_payload(f.read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', f'attachment; filename="{os.path.basename(adjunto_audio)}"')
        msg.attach(part)

    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(remitente, password)
        text = msg.as_string()
        server.sendmail(remitente, destinatario, text)
        server.quit()
        print(f"Correo enviado a {destinatario} con exito.")
    except Exception as e:
        print(f"Error al enviar el correo a {destinatario}: {e}")

def consolidar_pdfs(archivos_pdf, nombre_pdf_maestro):
    doc_maestro = fitz.open()
    for pdf in archivos_pdf:
        with fitz.open(pdf) as doc:
            doc_maestro.insert_pdf(doc)
    doc_maestro.save(nombre_pdf_maestro)
    doc_maestro.close()
    print(f"Todos los informes consolidados en {nombre_pdf_maestro}.")

In [12]:
archivos_informes_word = []
archivos_informes_pdf = []

for empleado in empleados:
    nombre_base = empleado.nombre.replace(' ', '_')
    nombre_grafico = f'grafico_{nombre_base}.png'
    nombre_informe_word = f'informe_{nombre_base}.docx'
    
    generar_grafico_radar(empleado, nombre_grafico)
    nombre_audio = crear_informe_word(empleado, nombre_informe_word, nombre_grafico)
    archivos_informes_word.append(nombre_informe_word)
    
    destinatario_ejemplo = 'villalobosdarwin349@gmail.com' 
    enviar_correo(destinatario_ejemplo, f'Informe de Desempeno {empleado.nombre}', 
                   f'Adjunto su informe de desempeno y un resumen en audio.', 
                   nombre_informe_word, nombre_audio)
    
    nombre_pdf = nombre_informe_word.replace('.docx', '.pdf')
    doc_temp = fitz.open()
    doc_temp.new_page()
    doc_temp.save(nombre_pdf)
    doc_temp.close()
    archivos_informes_pdf.append(nombre_pdf)

consolidar_pdfs(archivos_informes_pdf, 'informe_maestro.pdf')

Grafico de radar para Ana Gomez creado.
Informe de Word para Ana Gomez creado.
Resumen de audio para Ana Gomez creado.
Correo enviado a villalobosdarwin349@gmail.com con exito.
Grafico de radar para Luis Perez creado.
ATENCION: El informe de Luis Perez requiere una revision manual.
Informe de Word para Luis Perez creado.
Resumen de audio para Luis Perez creado.
Correo enviado a villalobosdarwin349@gmail.com con exito.
Grafico de radar para Sofia Ruiz creado.
Informe de Word para Sofia Ruiz creado.
Resumen de audio para Sofia Ruiz creado.
Correo enviado a villalobosdarwin349@gmail.com con exito.
Grafico de radar para Carlos Soto creado.
ATENCION: El informe de Carlos Soto requiere una revision manual.
Informe de Word para Carlos Soto creado.
Resumen de audio para Carlos Soto creado.
Correo enviado a villalobosdarwin349@gmail.com con exito.
Todos los informes consolidados en informe_maestro.pdf.
